In [1]:
include("../../ad_hoc/sample.jl")
using .SampleTable

include("../../../../utils/sqlite/v1/interval.jl")
using .IntervalTable

include("../../../../utils/sqlite/v1/twofold_interval.jl")
using .TwofoldIntervalTable

include("../../../../utils/sqlite/v1/sqlite.jl")
using .SQLiteHelper

include("./method1.jl")
using .Method1

In [2]:
M = 2
methodNum = 1
matrixTableName = "sample_interval_pcms"
sampleTableName = "samples"
resultTableName = "method$(methodNum)_results"

"method1_results"

In [3]:
for n = 5:5
    db = database("../../experiment_20240522_n$(n)_M$(M).sqlite")
    # dropTable(db, resultTableName)
    createMatrixTable(db, resultTableName)
end

In [4]:
function loadSamples(n::Int)
    db = database("../../experiment_20240522_n$(n)_M$(M).sqlite")
    df = parseAndReturnAllMatrixData(db, sampleTableName, matrixTableName)
end

loadSamples (generic function with 1 method)

In [5]:
@inline function processSamples(sampleData, n::Int)
    db = database("../../experiment_20240522_n$(n)_M$(M).sqlite")
    Threads.@threads for sample in sampleData
        sampleId = sample.sample_id
        matrices = sample.matrices
        try
            if length(matrices) >= 2
                A₁ = matrices[1].data
                A₂ = matrices[2].data
                𝒜 = method1(A₁, A₂)
                insertTwofoldIntervalData(db, resultTableName, sampleId, 𝒜)
            else
                display("Not enough matrices for sample $sampleId")
            end
        catch e
            display("Error occurred in sample $sampleId")
            display(e)
        end
    end
end

processSamples (generic function with 1 method)

In [6]:
for n = 5:5
    sampleData = loadSamples(n)
    processSamples(sampleData, n)
end

"Error occurred in sample 840eb68e-5e2d-4496-9466-c6467a9e347e"

ErrorException("Failed to consistentize a twofold interval PCM.")

"Error occurred in sample 09787bcd-5c22-465b-ae3e-b7d45b51d4b1"

ErrorException("Failed to consistentize a twofold interval PCM.")